Main classes to deal with:
- `SerafinHeader`
- `Read` (derived from `Serafin`)
- `Write` (derived from `Serafin`)

# Read Telemac file

Read a binary Selafin file.
Automatic dectection of **precision** (single or double) and **endianness** (big or little endian).

In [5]:
from pyteltools.slf import Serafin

with Serafin.Read('../scripts_PyTelTools_validation/data/Yen/fis_yen-exp.slf', 'en') as resin:
    # Read header (SerafinHeader is stored in `header` attribute of `Serafin` class)
    resin.read_header()
    
    # Display a summary
    print(resin.header.summary())
    
    # Get time (in seconds) and display it
    resin.get_time()
    print(resin.time)

Reading the input file: "../scripts_PyTelTools_validation/data/Yen/fis_yen-exp.slf" of size 2078184 bytes


The file is of type SERAPHIN 2D. It has 14 variables,
on 1894 nodes and 3093 elements for 19 time frames.
[0.0, 1000.0, 2000.0, 3000.0, 4000.0, 5000.0, 6000.0, 7000.0, 8000.0, 9000.0, 10000.0, 11000.0, 12000.0, 13000.0, 14000.0, 15000.0, 16000.0, 17000.0, 18000.0]


# Write Telemac file

In [2]:
import numpy as np

from pyteltools.slf import Serafin


with Serafin.Read('../scripts_PyTelTools_validation/data/Yen/fis_yen-exp.slf', 'en') as resin:
    resin.read_header()
    # Copy header
    output_header = resin.header.copy()
    
    # Change some header attributes if required
    #output_header.toggle_endianness()
    #output_header.to_single_precision()

    values = np.empty((output_header.nb_var, output_header.nb_nodes), dtype=output_header.np_float_type)
    with Serafin.Write('/tmp/test.slf', 'fr', overwrite=True) as resout:
        resout.write_header(output_header)

        # Copy all frames
        for time_index, time in enumerate(resin.time):
            for i, var_ID in enumerate(output_header.var_IDs):
                values[i, :] = resin.read_var_in_frame(time_index, var_ID)
            resout.write_entire_frame(output_header, time, values)

Reading the input file: "../scripts_PyTelTools_validation/data/Yen/fis_yen-exp.slf" of size 2078184 bytes
Writing the output file: "/tmp/test.slf"


# Handle exceptions

Some custom exceptions are defined in `pyteltools.slf.Serafin`:
* `SerafinRequestError`: Serafin file content checking
* `SerafinValidationError`: requesting invalid values from Serafin object